In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import utils
from scipy.optimize import least_squares
import numpy as np
import os
from scipy.integrate import quad
from scipy.stats import norm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Black-Scholes formula for option pricing
def BSFormula(S, K, t, r, vol, callPutFlag):
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * t) / (vol * np.sqrt(t))
    d2 = d1 - vol * np.sqrt(t)
    if callPutFlag == 1:  # Call option
        price = S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
    else:  # Put option
        price = K * np.exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

In [4]:
base_path = 'data/'

In [5]:
file_name = 'vixvol_20191220.csv'
file_path = os.path.join(base_path, file_name)
df = pd.read_csv(file_path, index_col=0)

In [6]:
# Goal: estimate H and \nu

In [7]:
def strip_vix_sq(_data, expiry: int):
    data = _data[_data.Expiry == expiry].dropna()
    f = data['Fwd'].iloc[0]
    integral = data['CallMid'].sum() * 2
    return f**2 + integral


def strip_vix_quartic(data):
    data['Fwd']**4 + (12*data['Strike']**2 * data['CallMid']).sum()

In [8]:
expirations = df['Expiry'].unique()

In [9]:
for e in expirations:
    print(strip_vix_sq(df, e))

179.17119295842068
217.0160529089184
236.3521621046104
247.74341129745918
271.9298800811591
350.6929692351954
353.1468451707987
384.20899205834746
407.43013044647523
413.76054689666535


In [10]:
def vix2(ivolData, expiry: int):
    df = ivolData[ivolData['Expiry'] == expiry]
    texp = df['Texp'].values[0]
    mask = ~df['Bid'].isna()
    df = df.loc[mask]

    midVol = 0.5 * (df['Ask'] + df['Bid'])
    fwd = df['Fwd'].values[0]  # Forward price
    strikes = df['Strike']
    k = np.log(strikes / fwd).values  # Log-moneyness
    kmin, kmax = min(k), max(k)
    minvo, maxvo = midVol[k == kmin].values[0], midVol[k == kmax].values[0]

    def volInterp(kout):
        """
        Interpolate vol between strikes;
        set constant outside strike range, use Stineman inside
        """
        if not isinstance(kout, np.ndarray):
            kout = np.array([kout])
        return np.where(
            kout < kmin, minvo,
            np.where(
                kout > kmax, maxvo,
                utils.stineman_interp(k, midVol.values, kout)
            )
        )
    
    # Integrand functions for calls and puts
    def cTilde(y):
        K = np.exp(y)
        vol = volInterp(y)
        price = BSFormula(S=1., K=K, t=texp, r=0, vol=vol, callPutFlag=1)
        return np.exp(y) * price
    
    def pTilde(y):
        K = np.exp(y)
        vol = volInterp(y)
        price = BSFormula(S=1., K=K, t=texp, r=0, vol=vol, callPutFlag=0)
        return np.exp(y) * price
    
    
    # Compute the integrals
    callIntegral, _ = quad(cTilde, 0, 10)
    putIntegral, _ = quad(pTilde, -10, 0)
    
    # Calculate the result
    # res = fwd ** 2 + 2 * (callIntegral + putIntegral)
    res = fwd ** 2 * (1 + 2 * (callIntegral + putIntegral))
    return res


def vix4(ivolData, expiry: int):
    df = ivolData[ivolData['Expiry'] == expiry]
    texp = df['Texp'].values[0]
    mask = ~df['Bid'].isna()
    df = df.loc[mask]

    midVol = 0.5 * (df['Ask'] + df['Bid'])
    fwd = df['Fwd'].values[0]  # Forward price
    strikes = df['Strike']
    k = np.log(strikes / fwd).values  # Log-moneyness
    kmin, kmax = min(k), max(k)
    minvo, maxvo = midVol[k == kmin].values[0], midVol[k == kmax].values[0]

    def volInterp(kout):
        """
        Interpolate vol between strikes;
        set constant outside strike range, use Stineman inside
        """
        if not isinstance(kout, np.ndarray):
            kout = np.array([kout])
        return np.where(
            kout < kmin, minvo,
            np.where(
                kout > kmax, maxvo,
                utils.stineman_interp(k, midVol.values, kout)
            )
        )
    
    # Integrand functions for calls and puts
    def cTilde(y):
        K = np.exp(y)
        vol = volInterp(y)
        price = BSFormula(S=1., K=K, t=texp, r=0, vol=vol, callPutFlag=1)
        return np.exp(3*y) * price
    
    def pTilde(y):
        K = np.exp(y)
        vol = volInterp(y)
        price = BSFormula(S=1., K=K, t=texp, r=0, vol=vol, callPutFlag=0)
        return np.exp(3*y) * price
    
    
    # Compute the integrals
    callIntegral, _ = quad(cTilde, 0, 10)
    putIntegral, _ = quad(pTilde, -10, 0)
    
    # Calculate the result
    # res = fwd ** 2 + 2 * (callIntegral + putIntegral)
    res = fwd ** 4 * (1 + 12 * (callIntegral + putIntegral))
    return res

In [11]:
vix2(df, expirations[0])

171.43062393500315

In [12]:
vix4(df, expirations[0])

33924.08597847841

In [13]:
vix2(df, expirations[0])**2

29388.458822744476

$$
-2 \log \mathbb{E}[\Delta VIX ^2] + \log\mathbb{E}[\Delta^2 VIX ^4]
$$
$$
-2 \log \Delta \mathbb{E}[VIX ^2] + \log \Delta^2 \mathbb{E}[ VIX ^4] = \sigma^2_{mkt}
$$

In [14]:
def _const_curve(t):
    return np.full(len(t), .234**2) if isinstance(t, np.ndarray) else .234**2

xi_1 = utils.VarianceCurve(_const_curve)

In [15]:
T = df['Texp'].values[-1]
T * 252

124.18891170431205

In [16]:
H = utils.Hurst(0.07)
eta = 1.9
volvol = 1.2287  # 1.9 * utils.c_h(H) * np.sqrt(H.h2) / 2
print(volvol)
print(1.9 * np.sqrt(H.h2) / (utils.c_h(H) * 2))

1.2287
1.2286731886461313


In [17]:
exp_vix2_mkt = vix2(df, expiry=expirations[-1]) * utils.DELTA / 10**4
exp_vix4_mkt = vix4(df, expiry=expirations[-1]) * utils.DELTA**2 / 10**8
sigma_mkt = utils.sigma_lognormal(exp_vix2_mkt, exp_vix4_mkt)

exp_vix2_jac = utils.expected_vix_sq(xi_1)
exp_vix4_jac = utils.moment2_vix_sq(xi_1, delta=utils.DELTA, volvol=volvol, hurst=H, T=T)
sigma_jac = utils.sigma_lognormal(exp_vix2_jac, exp_vix4_jac)

sigma_jim = eta**2 * T**H.h2 * utils.f_supH(utils.DELTA / T, H)

In [18]:
sigma_jim

1.045296187066289

In [19]:
sigma_jac

1.0589127555745712

In [20]:
sigma_mkt

1.7703220650106424